In [1]:
from abc import ABC

from gulpio2 import GulpDirectory
from pathlib import Path
from moviepy.editor import ImageSequenceClip, clips_array

from collections import defaultdict

import pickle
import pandas as pd
from pathlib import Path

import torch as t
import torch.nn as nn
import numpy as np
from omegaconf import OmegaConf
from typing import Any, Dict, List, Sequence, Union

from systems import EpicActionRecognitionSystem
from systems import EpicActionRecogintionDataModule

from utils.metrics import compute_metrics
from utils.actions import action_id_from_verb_noun
from scipy.special import softmax

from GPUtil import showUtilization as gpu_usage
from tqdm import tqdm

In [9]:
verb_noun_link = {}

verbs = pd.read_csv('../datasets/epic/labels/EPIC_100_verb_classes.csv')
verbs = pd.read_csv('../datasets/epic/labels/EPIC_100_noun_classes.csv')


rgb_train = GulpDirectory('../datasets/epic/gulp/rgb/p01_rgb_test/')

_, meta = rgb_train['P01_01_96']

verbs['key'][meta['verb_class']]

# for i, c in enumerate(tqdm(rgb_train)):
#     for _, meta in c:
#         if meta['verb'] in verb_noun_link:
#             verb_noun_link[meta['verb']].append(meta['noun'])
#         else:
#             verb_noun_link[meta['verb']] = [meta['noun']] 

'take'

In [10]:
xd = list(set(verb_noun_link['take']))

with open('../datasets/epic/labels/verb_noun.pkl', 'wb') as f:
    pickle.dump(verb_noun_link, f)

with open('../datasets/epic/labels/verb_noun_unique.pkl', 'wb') as f:
    pickle.dump({verb: list(set(verb_noun_link[verb])) for verb in verb_noun_link.keys()} , f)


In [3]:
with open('../datasets/epic/labels/verb_noun_unique.pkl', 'rb') as pkl:
    verb_noun = pickle.load(pkl)
    
with open('../datasets/epic/labels/verb_noun_classes_unique.pkl', 'rb') as pkl:
    verb_noun_classes = pickle.load(pkl)

In [8]:
for k,j in zip(verb_noun.keys(), verb_noun_classes.keys()):
    assert len(verb_noun[k]) == len(verb_noun_classes[j])

In [24]:
v = 5

def callback(verb_class):
    
    noun_list=verb_noun[list(verb_noun.keys())[verb_class]]
    noun_class=verb_noun_classes[list(verb_noun_classes.keys())[verb_class]]
    
    assert len(noun_list) == len(noun_class)
    
    return [{'label':i, 'value':j} for i,j in zip(noun_list,noun_class)]

In [25]:
list(verb_.keys())[v], callback(v)

('close',
 [{'label': 'box', 'value': 3},
  {'label': 'drawer', 'value': 6},
  {'label': 'rosemary', 'value': 8},
  {'label': 'freezer', 'value': 138},
  {'label': 'jar', 'value': 12},
  {'label': 'oregano', 'value': 15},
  {'label': 'lid', 'value': 145},
  {'label': 'powder', 'value': 19},
  {'label': 'cupboard', 'value': 21},
  {'label': 'bag', 'value': 22},
  {'label': 'olive', 'value': 23},
  {'label': 'cooker:slow', 'value': 152},
  {'label': 'top', 'value': 277},
  {'label': 'oven', 'value': 31},
  {'label': 'milk', 'value': 36},
  {'label': 'fridge', 'value': 38},
  {'label': 'bin', 'value': 40},
  {'label': 'bottle', 'value': 42},
  {'label': 'oil', 'value': 44},
  {'label': 'salt', 'value': 46},
  {'label': 'liquid:washing', 'value': 64},
  {'label': 'kettle', 'value': 83},
  {'label': 'container', 'value': 113}])

In [3]:
with open('../datasets/epic/labels/verb_noun_classes_narration_unique.pkl', 'rb') as pkl:
    xdxd = pickle.load(pkl)


In [4]:
xdxd

{3: [(3, 'P01_01_97'),
  (8, 'P01_18_31'),
  (46, 'P01_10_0'),
  (3, 'P01_109_332'),
  (23, 'P01_05_123'),
  (19, 'P01_105_554'),
  (3, 'P01_05_16'),
  (12, 'P01_09_494'),
  (8, 'P01_18_378'),
  (36, 'P01_105_383'),
  (46, 'P01_09_653'),
  (21, 'P01_18_288'),
  (8, 'P01_105_40'),
  (40, 'P01_109_180'),
  (3, 'P01_01_30'),
  (8, 'P01_05_43'),
  (95, 'P01_18_192'),
  (21, 'P01_18_276'),
  (8, 'P01_09_513'),
  (8, 'P01_09_605'),
  (44, 'P01_105_569'),
  (3, 'P01_09_597'),
  (12, 'P01_09_726'),
  (38, 'P01_01_161'),
  (15, 'P01_17_281'),
  (3, 'P01_18_28'),
  (83, 'P01_09_365'),
  (152, 'P01_105_31'),
  (15, 'P01_05_156'),
  (8, 'P01_01_111'),
  (3, 'P01_07_20'),
  (23, 'P01_09_47'),
  (8, 'P01_09_861'),
  (12, 'P01_107_3'),
  (3, 'P01_01_217'),
  (3, 'P01_09_18'),
  (12, 'P01_07_45'),
  (36, 'P01_02_110'),
  (12, 'P01_03_25'),
  (12, 'P01_18_200'),
  (3, 'P01_09_411'),
  (8, 'P01_105_168'),
  (3, 'P01_16_13'),
  (3, 'P01_103_14'),
  (36, 'P01_18_144'),
  (38, 'P01_09_377'),
  (36, 'P01_06